In [212]:
import pandas as pd
import camelot
import time
import numpy as np
import re
import glob 
import re

In [213]:
# Read in original pdf file using camelot and transform it into dataframes.

# tables = camelot.read_pdf("raw.pdf", flavor='lattice', pages='1-end')

In [214]:
# Save parsed dataframes into csv.

# for i, table in enumerate(tables):
#     table.df.to_csv(f"data/{i}.csv", index=False, header=False)

# Read in saved CSV files

In [215]:
# Read in all csv files and save as list of dataframes.

filenames = glob.glob("data/*.csv")
onlynumbers = [filename[5:-4] for filename in filenames]
numbers = sorted([int(num) for num in onlynumbers])

dataframes = []
for num in numbers:
    dataframes.append(pd.read_csv(f"data/{num}.csv"))

# Remove the first three tables, which are not real data, but an example tables from the document.
dataframes = dataframes[3:]

default_columns = dataframes[0].columns.str.replace(r'\n','', regex=True)
default_columns = [col for col in default_columns if col != 'NO']

# Strip all \n from cell values
# Handle Error Cases
# Standardize Column format
for idx, dataframe in enumerate(dataframes):
    dataframe.replace(r'\n','', regex=True, inplace=True)
    dataframe.columns = dataframe.columns.str.replace(r'\n','', regex=True)
    
    dataframe.rename(columns={
        "*허가일*변경일" : "허가및변경일"
    }, inplace=True)
    
    dataframe.rename(columns={
        "NO 대표자" : "대표자"
    }, inplace=True)
    
    try:
        dataframes[idx] = dataframe[default_columns]
    except Exception as e:
        error_msg = e
        
        if str(error_msg) == "\"['용량(MW)', '허가및변경일'] not in index\"":
            dataframe.rename(columns={
                "Unnamed: 6" : "허가및변경일",
                "용량허가및변경(MW)일" : "용량(MW)"
            }, inplace=True)
            dataframes[idx] = dataframe[default_columns]
        else:
            print(f"Problem with Columns in Dataframe {idx} : {error_msg}")

In [216]:
# Concatenate all tables into a single master df

df = pd.concat(dataframes, ignore_index=True)

# Create column names

df.columns = ['owner', 'company', 'address', 'type', 'capacity_MW', 'approval_date', 'prep_time', 'note']

# Remove all rows with no cell values

df = df[~df.isnull().all(axis=1)]

In [217]:
quote_missing = df.approval_date.isna() & df.prep_time.notnull()
df.loc[quote_missing, "approval_date"] = df.loc[quote_missing, "approval_date"].fillna("“")

# Replace '“', '\"' values with proper cell values.

for column in df.columns:
    missing_index = df[(df[column] == '“') | (df[column] == '\"') | (df[column] == '‘“')].index.values.astype(int)
    for index in missing_index:
        df[column][index] = df[column][index-1]

df = df[~df.isnull().all(axis=1)]
df.reset_index(drop=True, inplace=True)

In [218]:
# Some "owner" cells include company names. 
# If owner cell has two words and the company cell is empty,
# transfer the second word from the owner cell to the company cell.

company_in_owner_col_filt = (df['owner'].str.split().str.len() > 1) & df['company'].isna()

companies_and_owners = df.loc[company_in_owner_col_filt, 'owner'].str.split()

owners = companies_and_owners.apply(lambda l: l[0])
companies = companies_and_owners.apply(lambda l: l[1])

df.loc[company_in_owner_col_filt, 'owner'] = owners
df.loc[company_in_owner_col_filt, 'company'] = companies


# Some "address" cells include company names. 

# If the company cell is empty and address cell is not, transfer the company name from the address cell to the company cell.

company_in_address_col_filt = df['approval_date'].notnull() & df['address'].notnull() & df['owner'].notnull() & df['company'].isna()

companies_and_adddress = df.loc[company_in_address_col_filt, 'address'].str.split()

companies = companies_and_adddress.apply(lambda l: l[0])
addresses = companies_and_adddress.apply(lambda l: " ".join(l[1:]))

df.loc[company_in_address_col_filt, 'company'] = companies
df.loc[company_in_address_col_filt, 'address'] = addresses



# Combine overflowing rows into a single row.

In [219]:
def concat(v1, v2):
    result = ""
    if v1 is not np.nan:
        result += str(v1)
    if result:
        result += " "
    if v2 is not np.nan:
        result += str(v2)
    if not result:
        result = np.nan
    return result

In [220]:
def roll_up(df):
    for idx in reversed(df[df["approval_date"].isna()].index):
        df.loc[idx-1] = df.loc[idx-1].combine(df.loc[idx], concat)
        df.drop(index=idx, inplace=True)
    df.reset_index(drop=True, inplace=True)

In [221]:
roll_up(df)

# Replace NaN in note column to "신규" (newly approved)

### Check if note section is in "prep_time" column and move them into "note" column.

In [223]:
pd.options.display.max_colwidth = None
pd.options.display.max_rows = None

df.prep_time.fillna("", inplace=True)


note_in_prep = ~df.prep_time.str.contains(r"^(\d|-|\.|\s|‘|호기|년이?내?|개?월|'|""|\(|\)|[A-Z])+$|_", na=False) & df.note.isna()
df_note_in_prep = df[note_in_prep]
df_note_in_prep.drop([12], axis=0, inplace=True)

/var/folders/_c/dcjy1m9109bd_sq8l8j8zsdr0000gn/T/ipykernel_65599/3406238560.py:7: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  note_in_prep = ~df.prep_time.str.contains(r"^(\d|-|\.|\s|‘|호기|년이?내?|개?월|'|""|\(|\)|[A-Z])+$|_", na=False) & df.note.isna()
/var/folders/_c/dcjy1m9109bd_sq8l8j8zsdr0000gn/T/ipykernel_65599/3406238560.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_note_in_prep.drop([12], axis=0, inplace=True)


owner                company  \
199          김홍권               수완에너지(주)   
470     미쯔오카히데노리                 고원풍력발전   
471          윤병석                 당진에코파워   
479          조성철                내포그린에너지   
489          정의섭                   전주파워   
505          김성회          한 국 지 역 난 방공사   
506          기찬수        거 창 풍 력 발 전주식회사   
508          최종하                  세현에너지   
509          오재창                 세종그린파워   
510         구자숭           엘 아 이 케 이 파워    
515          조영대                  청라에너지   
516          조창현         씨 에 이 치 엔 지니어링   
568          조재민                 포천민자발전   
569          이윤영  동양시멘트  소성로  폐열회수 발전사업   
570          윤병석                 당진에코파워   
571          김태우                 한국남부발전   
575          문성환              삼양사  울산공장   
576          하영봉                GS  E&R   
584          최성용                부산정관에너지   
591         허화도                    영 발전   
592          류승보                  에임슨파워   
594           조석                    한수원   
595          유정준                SK  E&S   
596          신헌식                 한라오엠에스   
599   정 몽 익 ,정몽진             케이씨씨  김천공장   
634          NaN                 안동풍력발전   
1407         심재원                   인코테크   
1503         김재종                 한일풍력발전   
1688          사장                    한수원   
1689        김상태                   부선풍력    
1840          조석          현대일렉트릭앤에너지시스템   
1841         조언우                 빛솔라에너지   
1883          사장                 한국남동발전   
2055         최성용                부산정관에너지   
2165         박근범                엘아이케이파워   
2192          사장                    한수원   
2195         심재원                   인코테크   
2627         최민식                에프알피에너지   
2628         박근수                 고흥신에너지   
2629         임은상               디에스그린에너지   
2630         허용수                 지에스에너지   
2632         공유덕                  에너지워터   
2633         조성철                내포그린에너지   
2634         민병원                  청우E&S   
2635         전영수                   이엔테크   
2636         우혁재                 우앤장홀딩스   
2637         조경애                   위니스트   
2638         윤종근                 한국남부발전   
2640         심재원                   인코테크   
2641         박정학                 명성파워그린   
2642         박정학                 명성파워그린   
2643         박정학                 명성파워그린   
2644         박정학                 명성파워그린   
2645         박관숙              고성하이드로솔루션   
2646         구영임               홍천율전풍력발전   
2647         한정현                 정선풍력발전   
2648         김재갑                   영천풍력   
2649         조병련                청송면봉산풍력   
2650         한정현                 보령풍력발전   
2651         김기열               에이치에스에너지   
2652         최재영               경주강동풍력발전   
2653         신경철           이원신재생에너지복지마을   
2654         김동석                 아이티에너지   
2655         김동석                 아이티에너지   
2658         최우진              청사포해상풍력발전   

                                                                                         address  \
199                                                                             광주광역시 광산구수완로 130   
470                                                                    강원  태백시  창죽동  산  63번지  일원   
471                                                                충남  당진시  석문면  교로3리  1026번지  외   
479                                                                     충남  홍성군  홍북면,예산군 삽교읍  일원   
489                                                                         전북  전주시  덕진구  팔복로 59   
505                                                                            전남  나주시  산포면  세남로   
506                                                                  경남  거창군  신원면  덕산리 산57번지  일원   
508                                             충남  당진시  석문면  난지도리  183,  221-1,  223-2, 266-2일원   
509                                                                           세종시 장군면 봉안리 160-13   
510                                                                       강원  강릉시  옥계면  산계리 8번지    
515                                                                     

In [224]:
def retrieve_note(prep):
    note = re.findall(r"^\d{4}\.?-?\d{1,2}\.?-?\d{1,2}(.*)", prep)[0].strip()
    return note

df_note_in_prep.note = df_note_in_prep.prep_time.apply(lambda x: retrieve_note(x))

df.loc[df_note_in_prep.index, "note"] = df_note_in_prep.note

/var/folders/_c/dcjy1m9109bd_sq8l8j8zsdr0000gn/T/ipykernel_65599/4185455413.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_note_in_prep.note = df_note_in_prep.prep_time.apply(lambda x: retrieve_note(x))


In [225]:
df.note.fillna('신규', inplace=True)
df.note = df.note.str.replace(" ", "") \
    .str.replace("nan", "신규") \
    .str.replace("nannan", "신규") \
    .str.replace("신규신규", "신규")


# Check non-Nan values in note to see if they are also onewly approved.

not_new = ['연장', '변경', '취득', '용량', '용량변경', '정정', '재발급', '인가', '양수', '추가', '합병', '증설', '양도', '개시', '반납', '취소', '축소', '준비기간', '조정', '법인', 'SRF', "→", '지번', '재교부', '전환', '수정', '발전', '종류', '공급', '폐기물', '배방읍', '구분']
# new = ['신규', '이내', '허가', '일부', '동의']

df.loc[~df.note.str.contains('|'.join(not_new)), "note"] = "신규"


In [226]:
pd.options.display.max_colwidth=20

df[~df.note.str.contains('|'.join(not_new))].note.unique()

array(['신규'], dtype=object)

# Convert "approval_date" from df_new to datetime

In [227]:
# Only select plants that are newly approved

df_new = df[df.note=='신규']

In [228]:
# Clean up approval_date

df_new.approval_date = df_new.approval_date.str.replace("`", "20", regex=False).str.replace("‘", "20", regex=False).str.replace("'", "20", regex=False).str.replace(".", "-", regex=False)
df_new.approval_date = df_new.approval_date \
                        .str.replace(r"^12-5$", "2011-12-05", regex=True) \
                        .str.replace(r"^3-5$", "2012-03-05", regex=True) \
                        .str.replace(r"^3-30$", "2012-03-30", regex=True) \
                        .str.replace(r"^4-26$", "2012-04-26", regex=True) \
                        .str.replace(r"^12-6-4$", "2012-06-04", regex=True)


/var/folders/_c/dcjy1m9109bd_sq8l8j8zsdr0000gn/T/ipykernel_65599/2103708735.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new.approval_date = df_new.approval_date.str.replace("`", "20", regex=False).str.replace("‘", "20", regex=False).str.replace("'", "20", regex=False).str.replace(".", "-", regex=False)
/var/folders/_c/dcjy1m9109bd_sq8l8j8zsdr0000gn/T/ipykernel_65599/2103708735.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new.approval_date = df_new.approval_date \


In [229]:
df_new['year'] = df_new.approval_date.str[:4]

month_regex = r"\d{4}-(\d{1,2})"

def string_to_month(string):
    month = re.findall(month_regex, string)[0]
    return month

df_new['month'] = df_new.approval_date.apply(lambda x: string_to_month(x))
df_new['month'] = df_new['month'].str.zfill(2)

df_new['cleaned_approval_time'] = df_new['year'] + "-" + df_new['month']
df_new['cleaned_approval_time'] = pd.to_datetime(df_new['cleaned_approval_time'], format="%Y-%m")

/var/folders/_c/dcjy1m9109bd_sq8l8j8zsdr0000gn/T/ipykernel_65599/1993496021.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['year'] = df_new.approval_date.str[:4]
/var/folders/_c/dcjy1m9109bd_sq8l8j8zsdr0000gn/T/ipykernel_65599/1993496021.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['month'] = df_new.approval_date.apply(lambda x: string_to_month(x))
/var/folders/_c/dcjy1m9109bd_sq8l8j8zsdr0000gn/T/ipykernel_65599/1993496021.py:10: SettingWithCopyWarning: 
A value is trying to be set

# Clean up "용량(MW)" (capacity) column into a consistent format

### 1. If the cell is in KW, change the number and unit to MW.

In [230]:
df_in_kw = df_new[df_new['capacity_MW'].str.lower().str.contains("kw", na=False)]

# Identify which cells have multiple plants, so I can multiply the capacity by two later on.
has_multiple_plants = df_in_kw['capacity_MW'].str.contains("기")

# df.capacity_MW["kw" in df.capacity_MW.str.lower()]
has_10k = df_in_kw['capacity_MW'].str.contains("만")
has_1k = df_in_kw['capacity_MW'].str.contains("천")

df_in_kw.loc[has_10k, 'cleaned_capacity'] = df_in_kw['capacity_MW'].str.replace("만", "0000")
df_in_kw.loc[has_10k & has_1k, 'cleaned_capacity'] = df_in_kw['capacity_MW'].str.replace("만", "").str.replace("천", "000")

# Fill in empty 'cleaned_capacity' column with 'capacity_MW' values.
df_in_kw.cleaned_capacity = df_in_kw.cleaned_capacity.fillna(df_in_kw.capacity_MW)

# Convert capacity value into int and divide capacity by 1,000
df_in_kw.cleaned_capacity = df_in_kw.cleaned_capacity \
                                .str.lower() \
                                .str.replace("총", "") \
                                .str.replace("kw", "") \
                                .str.replace(",", "") \
                                .str.replace("7-8호기", "") \
                                .str.replace("2기", "")

df_in_kw.cleaned_capacity = df_in_kw.cleaned_capacity.astype(float)
df_in_kw.cleaned_capacity = df_in_kw.cleaned_capacity / 1000
df_in_kw[has_multiple_plants].cleaned_capacity = df_in_kw[has_multiple_plants].cleaned_capacity * 2


df_in_kw['capacity_MW'] = df_in_kw['cleaned_capacity']
df_in_kw.drop(columns=['cleaned_capacity'], inplace=True)

df_new.loc[df_in_kw.index, 'capacity_MW'] = df_in_kw['capacity_MW']


/var/folders/_c/dcjy1m9109bd_sq8l8j8zsdr0000gn/T/ipykernel_65599/1890943783.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_in_kw.loc[has_10k, 'cleaned_capacity'] = df_in_kw['capacity_MW'].str.replace("만", "0000")
/var/folders/_c/dcjy1m9109bd_sq8l8j8zsdr0000gn/T/ipykernel_65599/1890943783.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_in_kw.cleaned_capacity = df_in_kw.cleaned_capacity.fillna(df_in_kw.capacity_MW)
/var/folders/_c/dcjy1m9109bd_sq8l8j8zsdr0000gn/T/ipykernel_65599/1890943783.py

### 2. Check if there are other problems with the capacity_MW columns

In [231]:
# Remove all commas and "MW" from values.
df_new.capacity_MW = df_new.capacity_MW.astype(str)\
                        .str.lower() \
                        .str.replace(",", "") \
                        .str.replace("mw", "") \
                        .str.replace("nan", "") \
                        .str.replace("!", "1") \
                        .str.replace(r"×?1기", "")

# Remove additional information from capacity_MW cells.
has_additional_info = df_new.capacity_MW.str.contains("\(") & df_new.capacity_MW.str.contains("\)")
df_new["capacity_MW"] = df_new["capacity_MW"].str.replace(r"[(].*[)]", "", regex=True)

# Additional manual cleaning for misread values by cross-referncig them with original pdf file.
df_new.capacity_MW = df_new.capacity_MW.str.replace("13900×22500", "36400") \
                                        .str.replace("834.3\(gt2기 st", "834.3") \
                                        .str.replace("13500×5기1000×3기380", "70880") \
                                        .str.replace("여수250군장122.9", "372.9") \
                                        .str.replace("126313.67.3", "1283.9") \
                                        .str.replace("555.17.2", "762.3") \
                                        .str.replace("19.99 3.0", "19.993") \
                                        .str.replace(" ", "") \
                                        .str.strip() \
                                        .str.replace("82219.2", "841.2") \

# There are three entries with empty capacity, where its 9.9 capacity is placed within the energy type. Fix these entires.
df_new.loc[df_new.capacity_MW == "", 'capacity_MW'] = df_new[df_new.capacity_MW == ""]["capacity_MW"].str.replace("", "9.9")


# Some capacity numbers are unreasonbly high, and may actually be in kw. Identify these entries.
df_new.capacity_MW = df_new.capacity_MW.astype(float)
df_new[df_new.capacity_MW>1000]
df_new.loc[df_new.capacity_MW >= 35000, 'capacity_MW'] = df_new[df_new.capacity_MW >= 35000]["capacity_MW"]/1000



/var/folders/_c/dcjy1m9109bd_sq8l8j8zsdr0000gn/T/ipykernel_65599/3657384742.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  .str.replace(r"×?1기", "")
/var/folders/_c/dcjy1m9109bd_sq8l8j8zsdr0000gn/T/ipykernel_65599/3657384742.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new.capacity_MW = df_new.capacity_MW.astype(str)\
/var/folders/_c/dcjy1m9109bd_sq8l8j8zsdr0000gn/T/ipykernel_65599/3657384742.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


# Clean "Type" from df_new

In [232]:
df_new.type = df_new.type.str.replace(" ", "")

# Manual cleaning misread values by cross-referencing with original pdf.
df_new.company = df_new.company.str.replace("영암태 전", "영암태양광발전")
df_new.address = df_new.address.str.replace("양광발 전", "전")
df_new.loc[956, "type"] = "태양광"
df_new.loc[1667, "type"] = "태양광"
df_new.loc[2572, "type"] = "풍력"

/var/folders/_c/dcjy1m9109bd_sq8l8j8zsdr0000gn/T/ipykernel_65599/3518237327.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new.type = df_new.type.str.replace(" ", "")
/var/folders/_c/dcjy1m9109bd_sq8l8j8zsdr0000gn/T/ipykernel_65599/3518237327.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new.company = df_new.company.str.replace("영암태 전", "영암태양광발전")
/var/folders/_c/dcjy1m9109bd_sq8l8j8zsdr0000gn/T/ipykernel_65599/3518237327.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy 

### Simplify type into "cleaned_type" column.

In [233]:
df_new['cleaned_type'] = np.nan

# df_new.loc[df_new.type.str.contains("열병합", na=False), 'cleaned_type'] = "coal"
# df_new.loc[df_new.type.str.contains("복합", na=False), 'cleaned_type'] = "coal"
df_new.loc[df_new.type.str.contains("가스", na=False), 'cleaned_type'] = "gas"
df_new.loc[df_new.type.str.contains("폐열", na=False), 'cleaned_type'] = "coal"
df_new.loc[df_new.type.str.contains("바이오", na=False), 'cleaned_type'] = "bioenergy"
df_new.loc[df_new.type.str.contains("목질계", na=False), 'cleaned_type'] = "bioenergy"
df_new.loc[df_new.type.str.contains("우드칩", na=False), 'cleaned_type'] = "bioenergy"
df_new.loc[df_new.type.str.contains("원자력", na=False), 'cleaned_type'] = "nuclear"
df_new.loc[df_new.type.str.contains("수력", na=False), 'cleaned_type'] = "hydro"
df_new.loc[df_new.type.str.contains("양수", na=False), 'cleaned_type'] = "hydro"
df_new.loc[df_new.type.str.contains("풍력", na=False), 'cleaned_type'] = "wind"
df_new.loc[df_new.type.str.contains("태양광", na=False), 'cleaned_type'] = "solar"
df_new.loc[df_new.type.str.contains("석탄", na=False), 'cleaned_type'] = "coal"
df_new.loc[df_new.type.str.contains("유연탄", na=False), 'cleaned_type'] = "coal"
df_new.loc[df_new.type.str.lower().str.contains("ng", na=False), 'cleaned_type'] = "gas"
# df_new.loc[df_new.type.str.lower().str.contains("srf", na=False), 'cleaned_type'] = "srf"
# df_new.loc[df_new.type.str.lower().str.contains("생활", na=False), 'cleaned_type'] = "srf"
# df_new.loc[df_new.type.str.lower().str.contains("폐기물", na=False), 'cleaned_type'] = "srf"
# df_new.loc[df_new.type.str.lower().str.contains("페기물", na=False), 'cleaned_type'] = "srf"
# df_new.loc[df_new.type.str.lower().str.contains("고체연료", na=False), 'cleaned_type'] = "srf"
# df_new.loc[df_new.type.str.lower().str.contains("고형연료", na=False), 'cleaned_type'] = "srf"
# df_new.loc[df_new.type.str.lower().str.contains("연료전지", na=False), 'cleaned_type'] = "hydrogen fuel cells"
# df_new.loc[df_new.type.str.lower().str.contains("조력", na=False), 'cleaned_type'] = "ocean"
# df_new.loc[df_new.type.str.lower().str.contains("해양", na=False), 'cleaned_type'] = "ocean"
# df_new.loc[df_new.type.str.lower().str.contains("스팀", na=False), 'cleaned_type'] = "steam"
# df_new.loc[df_new.type.str.lower().str.contains("증기", na=False), 'cleaned_type'] = "steam"
df_new.loc[df_new.cleaned_type.isna(), 'cleaned_type'] = "others"




/var/folders/_c/dcjy1m9109bd_sq8l8j8zsdr0000gn/T/ipykernel_65599/2481554190.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['cleaned_type'] = np.nan


In [234]:
df_cleaned = df_new[['cleaned_type', 'capacity_MW', 'address', 'year', 'company', 'owner']]

# Clean up "address" column

### Check repeating addresses to make sure they are not redundancies.

Following addresses are checked with original pdf to prove they are separate plants:
* 강원도  영월군  영월읍  정양리  540,  541-1,  541-2, 566,  567,  568,  569
* 전남 나주시 영산동664-1 등지 (Multiple parks by same company in the same location (likely expansion)
* 인천시  남구  학익동  723번지
* 강원도  영월군  상동읍  천평리  249번지  일원
* 강원  정선군  고한읍  고한리 산216-1  등지
* 전남  광양시  광양읍  세풍리 2200
* 충남  보령시  성주면  개화리 산30 
* 울산시 북구 효암로 84-13 
* 경북 김천 어모면
* 경북  의성군  의성읍  철파리  531,  532-2,  533-2, 530-1,  530-2  번지 
* 경기 포천 창수면 
* 경북  안동시  임하면  신덕리  산93,  96,  97,  100-1, 100-2 
* 인천시  서구  백범로  934번길 23
* 부산 강서구 신호동
* 경남  함안군  군북면  장지리 1407
* 강원도  태백시  동점동  동 점산업단지  블록  3-3
* 경기도  양주시  남면  삼일로485번길  67-22
* 강원도  태백시  동점동  동점산업단지  블록  3-6(동점산업단지내) 
* 충남  서산시  대산읍  죽엽로 397일원(대죽리)
* 부산시 기장군 정관면 산단1로 83
* 전남 곡성군 오곡면 침곡리 산39, 송정리 산96 일원
* 충남 아산시 둔포면 염작리 92-10, 92-28
* 충남 당진군 석문면교로리 산 974	
* 인천시 서구 백석동 58  
* 경기도  남양주  왕숙  공공주택지구
* 경남 양산시 원동면대리 산 93-4
* 경남 양산시 원동면대리 산 93-4	

Following addresses are pending investigations.
* 충남  아산시  둔포면  염작리  92-28 -> I have reached out to KOREC to clarify the situation. The data has not be treated and all addresses remain in df_new.
* 제주시  구좌읍  덕천리  산82 (Are these three separate solar farms? Or just duplication by mistake?)
* 전북  익산시  석암로  3길 80(익산제2일반산업단지 내) (Are these three separate solar farms? Or just duplication by mistake?)
* 경기  양주시  남면  상수리  산112-11
* 경북  영덕군  영덕읍  매정리  1162번지  일원
* 광주시  북구  운정동  627번지외  5필지
* 전남  장흥군  장흥읍  삼산리  749-1번지  일원
* 경북 군위군 삼국유사면 화수리 산45-1, 산44

Except for the pending investigations, all addresses should be considered as unique from here below.

In [235]:
# Some company column includes addresses. Fix these rows.

no_address = df_cleaned.address.isna()

# df_cleaned[no_address].address = df_cleaned[no_address].company.apply(lambda x: company_to_address(x))
df_cleaned[no_address].company.str.split(" ")
# df_cleaned.loc[no_address, 'address'] = df_

companies_and_adddress = df_cleaned.loc[no_address, 'company'].str.split()

companies = companies_and_adddress.apply(lambda l: l[0])
addresses = companies_and_adddress.apply(lambda l: " ".join(l[1:]))

df_cleaned.loc[no_address, 'company'] = companies
df_cleaned.loc[no_address, 'address'] = addresses

### Identify coordinate location entries.

In [236]:
has_coord = df_cleaned.address.str.contains("°|˚")

address_and_coord = df_cleaned.loc[has_coord, 'address'].str.split('\(')
addresses = address_and_coord.apply(lambda x: x[0])
df_cleaned.loc[has_coord, 'address'] = addresses
df_cleaned.loc[1424, 'address'] = "전남 신안군 도초면 우이도리 남동측 해상"

In [260]:
pd.options.display.max_colwidth = None

# Clean other rows with coordinates in the cell.

df_cleaned[df_cleaned.address.str.contains(r"\d{8}")]
address_and_coord = df_cleaned.loc[df_cleaned.address.str.contains(r"\d{8}"), 'address'].str.split('\(')
addresses = address_and_coord.apply(lambda x: x[0])
df_cleaned.loc[df_cleaned.address.str.contains(r"\d{8}"), 'address'] = addresses

df_cleaned[df_cleaned.address.str.contains("N")]
address_and_coord = df_cleaned.loc[df_cleaned.address.str.contains("N"), 'address'].str.split('\(')
addresses = address_and_coord.apply(lambda x: x[0])
df_cleaned.loc[df_cleaned.address.str.contains("N"), 'address'] = addresses

has_coord = df_cleaned.address.str.contains("위도") & df_cleaned.address.str.contains("경도")
df_cleaned[has_coord]
address_and_coord = df_cleaned.loc[has_coord, 'address'].str.split('\(')
addresses = address_and_coord.apply(lambda x: x[0])
df_cleaned.loc[has_coord, 'address'] = addresses

df_cleaned.address = df_cleaned.address.str.strip()

/var/folders/_c/dcjy1m9109bd_sq8l8j8zsdr0000gn/T/ipykernel_65599/1666838140.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned.address = df_cleaned.address.str.strip()


In [267]:
df_cleaned.capacity_MW = df_cleaned.capacity_MW.round(1)
df_cleaned.to_csv("df_cleaned.csv", index=False)

/var/folders/_c/dcjy1m9109bd_sq8l8j8zsdr0000gn/T/ipykernel_65599/3502554693.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned.capacity_MW = df_cleaned.capacity_MW.round(1)
